In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings("ignore")
import sys
import os

### 1. Load Data

In [5]:
tickers = ['TSLA', 'BND', 'SPY']
start_date = '2015-07-01'
end_date = '2025-07-31'
data =yf.download(tickers, start=start_date, end=end_date)

[*********************100%***********************]  3 of 3 completed


In [7]:
data.shape

(2535, 15)

In [ ]:
## Save the dataset
data.to_csv('../data/raw/hist_finance_data.csv')

In [9]:
sys.path.append(os.path.abspath('../src/'))

In [ ]:
from preprocess import data_loader
df = load_data('../data/raw/historic')

### 2. Data Cleaning

In [ ]:

for ticker in tickers:
    df = data[ticker]
    # Ensure no missing values
    if df.isnull().any().any():
        df = df.fillna(method='ffill')  # Forward fill for missing values
    # Verify data types
    df[['Open', 'High', 'Low', 'Close', 'Adj Close']] = df[['Open', 'High', 'Low', 'Close', 'Adj Close']].astype(float)
    df['Volume'] = df['Volume'].astype(int)
    data[ticker] = df.sort_index()

### 3. Descriptive Statistics

In [ ]:
stats = {ticker: data[ticker][['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].describe() for ticker in tickers}
for ticker, stat in stats.items():
    print(f"\n{ticker} Statistics:")
    print(stat)

### 4. Calculate daily returns

In [ ]:
for ticker in tickers:
    data[ticker]['Daily Return'] = data[ticker]['Adj Close'].pct_change()
    print(f"\n{ticker} Daily Return Statistics:")
    print(data[ticker]['Daily Return'].describe())